## Camera Calibration

In [16]:
# import packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from moviepy.editor import VideoFileClip
%matplotlib inline

CHESSBOARD_WIDTH = 9
CHESSBOARD_HEIGHT = 6
OUTPUT_DIR = 'output_images'
CALIBRATION_IMAGE_PATH = 'camera_cal'

In [17]:
%run utilities.ipynb

In [18]:
def calibrate():
    """
    Calibrate camera
    """
    calibration_images = read_images(CALIBRATION_IMAGE_PATH)
    obj_points, img_points = _extract_points(calibration_images,
                                             CHESSBOARD_WIDTH,
                                             CHESSBOARD_HEIGHT)
    # ret, return value
    # mtx, camera matrix used for transforming object points in 3d to ones in 2d
    # dist, distortion coefficient
    # rvecs, rotation vectors
    # tvecs, translation vectors
    return cv2.calibrateCamera(obj_points, 
                               img_points, 
                               get_shape_of_images(calibration_images), 
                               None, 
                               None)

def undistort_all(images, mtx, dist):
    """
    Undistort images
    """
    recovered_images = {}
    for fname, image in images.items():
        recovered_images[fname] = cv2.undistort(image, mtx, dist, None, mtx)
    return recovered_images

def undistort_one(image, mtx, dist):
    return cv2.undistort(image, mtx, dist, None, mtx)

def _extract_points(images, width, height, mark_image=False):
    """
    Return a list of image points and object points
    Parameters:
        images, image_name, image
        width, chessboard width
        height, chessboard height
        mark, mark corners on the images or not
    """
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    obj_point = np.zeros((width*height,3), np.float32)
    obj_point[:,:2] = np.mgrid[0:width, 0:height].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    obj_points = [] # 3d points in real world space
    img_points = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for _, img in images.items():
        # turn the image into grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (width,height), None)
        # If found, add object points, image points
        if ret == True:
            obj_points.append(obj_point)
            img_points.append(corners)
            if mark_image:
                # Draw corners on the image
                cv2.drawChessboardCorners(img, (width,height), corners, ret)
    return obj_points, img_points